In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
import numpy as np

In [2]:
# Memastikan penggunaan GPU jika tersedia
print("CUDA Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU Device Name:", torch.cuda.get_device_name(0))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Hyperparameters
batch_size = 128
num_classes = 10

# Transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load dataset
train_dataset = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

CUDA Available: True
GPU Device Name: NVIDIA GeForce GTX 1660
Using device: cuda


In [3]:
# CNN Model
class CNN(nn.Module):
    def __init__(self, kernel_size, pooling_type):
        super(CNN, self).__init__()
        
        if pooling_type == 'max':
            self.pooling = nn.MaxPool2d(kernel_size=2, stride=2)
        elif pooling_type == 'avg':
            self.pooling = nn.AvgPool2d(kernel_size=2, stride=2)

        self.conv1 = nn.Conv2d(1, 32, kernel_size=kernel_size, padding=kernel_size // 2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=kernel_size, padding=kernel_size // 2)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pooling(torch.relu(self.conv1(x)))
        x = self.pooling(torch.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
# Training Function
def train_model(model, optimizer, scheduler, criterion, num_epochs=5):
    early_stop_patience = 10
    best_loss = float('inf')
    patience_counter = 0

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        epoch_loss = running_loss / len(train_loader)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")

        # Validation Loss
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

        val_loss /= len(test_loader)
        scheduler.step(val_loss)

        # Early Stopping
        if val_loss < best_loss:
            best_loss = val_loss
            patience_counter = 0
        else:
            patience_counter += 1

        if patience_counter >= early_stop_patience:
            print("Early stopping triggered!")
            break

    return model

In [5]:
# Experiment Setup
kernel_sizes = [3, 5, 7]
pooling_types = ['max', 'avg']
epochs = [5, 50, 100, 250, 350]
optimizers = ['SGD', 'RMSProp', 'Adam']

# Results Storage
results = []
criterion = nn.CrossEntropyLoss()

for kernel_size in kernel_sizes:
    for pooling_type in pooling_types:
        for epoch in epochs:
            for optimizer_name in optimizers:
                print(f"\nRunning: Kernel Size={kernel_size}, Pooling={pooling_type}, Epochs={epoch}, Optimizer={optimizer_name}")
                
                model = CNN(kernel_size, pooling_type).to(device)

                if optimizer_name == 'SGD':
                    optimizer = optim.SGD(model.parameters(), lr=0.01)
                elif optimizer_name == 'RMSProp':
                    optimizer = optim.RMSprop(model.parameters(), lr=0.001)
                elif optimizer_name == 'Adam':
                    optimizer = optim.Adam(model.parameters(), lr=0.001)

                scheduler = ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.5)

                trained_model = train_model(model, optimizer, scheduler, criterion, num_epochs=epoch)

                # Evaluate Model
                trained_model.eval()
                test_loss = 0.0
                correct = 0
                total = 0
                with torch.no_grad():
                    for images, labels in test_loader:
                        images, labels = images.to(device), labels.to(device)
                        outputs = trained_model(images)
                        loss = criterion(outputs, labels)
                        test_loss += loss.item()
                        _, predicted = torch.max(outputs, 1)
                        total += labels.size(0)
                        correct += (predicted == labels).sum().item()

                accuracy = 100 * correct / total
                results.append({
                    'kernel_size': kernel_size,
                    'pooling': pooling_type,
                    'epochs': epoch,
                    'optimizer': optimizer_name,
                    'test_loss': test_loss / len(test_loader),
                    'accuracy': accuracy
                })

# Print Results
for result in results:
    print(result)


Running: Kernel Size=3, Pooling=max, Epochs=5, Optimizer=SGD
Epoch 1/5, Loss: 1.2596
Epoch 2/5, Loss: 0.5803
Epoch 3/5, Loss: 0.5091
Epoch 4/5, Loss: 0.4607
Epoch 5/5, Loss: 0.4328

Running: Kernel Size=3, Pooling=max, Epochs=5, Optimizer=RMSProp
Epoch 1/5, Loss: 0.4510
Epoch 2/5, Loss: 0.2789
Epoch 3/5, Loss: 0.2317
Epoch 4/5, Loss: 0.2012
Epoch 5/5, Loss: 0.1755

Running: Kernel Size=3, Pooling=max, Epochs=5, Optimizer=Adam
Epoch 1/5, Loss: 0.4890
Epoch 2/5, Loss: 0.3157
Epoch 3/5, Loss: 0.2694
Epoch 4/5, Loss: 0.2380
Epoch 5/5, Loss: 0.2146

Running: Kernel Size=3, Pooling=max, Epochs=50, Optimizer=SGD
Epoch 1/50, Loss: 1.2517
Epoch 2/50, Loss: 0.6117
Epoch 3/50, Loss: 0.5353
Epoch 4/50, Loss: 0.4889
Epoch 5/50, Loss: 0.4523
Epoch 6/50, Loss: 0.4249
Epoch 7/50, Loss: 0.4040
Epoch 8/50, Loss: 0.3895
Epoch 9/50, Loss: 0.3737
Epoch 10/50, Loss: 0.3601
Epoch 11/50, Loss: 0.3523
Epoch 12/50, Loss: 0.3415
Epoch 13/50, Loss: 0.3346
Epoch 14/50, Loss: 0.3254
Epoch 15/50, Loss: 0.3189
Epoch